In [ ]:
# start coding here

In [ ]:
import pypsa
import matplotlib.pyplot as plt
import geopandas as gpd
import numpy as np
import pandas as pd
from pathlib import Path
import seaborn as sns
from datetime import datetime
from cartopy import crs as ccrs
from pypsa.plot import add_legend_circles, add_legend_lines, add_legend_patches

In [ ]:
sns.set_theme("paper", style="darkgrid")

In [ ]:
n = pypsa.Network(snakemake.input.network)
regions_onshore = gpd.read_file(snakemake.input.regions_onshore)

In [ ]:
# TODO
n.carriers.loc["wind", "color"] = "lightblue"
n.carriers.loc["ng", "color"] = "indianred"

# Capacity plots

In [ ]:
if snakemake.wildcards.interconnect != "usa":
    bus_scale = 1e5
else:
    bus_scale = 5e3

line_scale = 2e3

In [ ]:
fig, ax = plt.subplots(figsize=(8, 8), subplot_kw={"projection": ccrs.PlateCarree()})
n.plot(
    bus_sizes=n.generators.groupby(["bus", "carrier"]).p_nom.sum() / bus_scale,
    bus_alpha=0.7,
    line_widths=n.lines.s_nom / line_scale,
    line_colors="teal",
    ax=ax,
    margin=0.2,
    color_geomap=True,
)
regions_onshore.plot(
    ax=ax,
    facecolor="whitesmoke",
    edgecolor="grey",
    aspect="equal",
    transform=ccrs.PlateCarree(),
    linewidth=0.1,
)

legend_kwargs = {"loc": "upper left", "labelspacing": 1.5, "frameon": False}
bus_sizes = [1000, 2000, 5000]  # in MW
line_sizes = [2000, 5000]  # in MW
add_legend_circles(
    ax,
    [s / bus_scale for s in bus_sizes],
    [f"{s / 1000} GW" for s in bus_sizes],
    legend_kw={"bbox_to_anchor": (1, 1), **legend_kwargs},
)
add_legend_lines(
    ax,
    [s / line_scale for s in line_sizes],
    [f"{s / 1000} GW" for s in line_sizes],
    legend_kw={"bbox_to_anchor": (1, 0.8), **legend_kwargs},
)
add_legend_patches(
    ax,
    n.carriers.color,
    n.carriers.nice_name,
    legend_kw={"bbox_to_anchor": (1, 0.5), **legend_kwargs},
)
fig.tight_layout()
fig.savefig(snakemake.output.capacity_map)

In [ ]:
fig, ax = plt.subplots()
capacities = n.generators.groupby("carrier").p_nom_opt.sum()
capacities.rename(n.carriers.nice_name, inplace=True)
colors = n.carriers.set_index("nice_name").color[capacities.index]
capacities.div(1e3).plot.bar(color=colors, ax=ax)
ax.set_ylabel("Total capacity [GW]")
ax.set_xlabel("")
fig.savefig(snakemake.output.capacity_bar)

# Operation plots

In [ ]:
if snakemake.wildcards.interconnect != "usa":
    bus_scale = 5e7
else:
    bus_scale = 1e5

line_scale = 1e6

In [ ]:
fig, ax = plt.subplots(figsize=(8, 8), subplot_kw={"projection": ccrs.PlateCarree()})
n.plot(
    bus_sizes=n.generators_t.p.sum()
    .groupby([n.generators.bus, n.generators.carrier])
    .sum()
    / bus_scale,
    bus_alpha=0.7,
    line_widths=n.lines_t.p0.sum() / line_scale,
    line_colors="teal",
    ax=ax,
    margin=0.2,
    color_geomap=True,
)
regions_onshore.plot(
    ax=ax,
    facecolor="whitesmoke",
    edgecolor="grey",
    aspect="equal",
    transform=ccrs.PlateCarree(),
    linewidth=0.1,
)

legend_kwargs = {"loc": "upper left", "labelspacing": 1.5, "frameon": False}
bus_sizes = [1000000, 2000000, 5000000]  # in MW
line_sizes = [2000000, 5000000]  # in MW
add_legend_circles(
    ax,
    [s / bus_scale for s in bus_sizes],
    [f"{s / 1e6} TWh" for s in bus_sizes],
    legend_kw={"bbox_to_anchor": (1, 1), **legend_kwargs},
)
add_legend_lines(
    ax,
    [s / line_scale for s in line_sizes],
    [f"{s / 1e6} TWh" for s in line_sizes],
    legend_kw={"bbox_to_anchor": (1, 0.8), **legend_kwargs},
)
add_legend_patches(
    ax,
    n.carriers.color,
    n.carriers.nice_name,
    legend_kw={"bbox_to_anchor": (1, 0.5), **legend_kwargs},
)
fig.tight_layout()
fig.savefig(snakemake.output.operation_map)

# Cost plots

In [ ]:
carriers = n.generators.carrier
production = (
    n.generators_t.p.groupby(carriers, axis=1)
    .sum()
    .rename(columns=n.carriers.nice_name)
    / 1e3
)
production = production.loc[:, production.sum() > 0.1]
demand = n.loads_t.p.sum(1).rename("Demand") / 1e3
colors = n.carriers.set_index("nice_name").color[production.columns]

for timeslice in list(range(1, 12)) + ["all"]:
    snapshots = (
        n.snapshots.get_loc(f"2016-{timeslice}")
        if timeslice != "all"
        else slice(None, None)
    )
    fig, ax = plt.subplots(figsize=(14, 4))
    production[snapshots].plot.area(ax=ax, color=colors, alpha=0.7, legend="reverse")
    # demand.plot.line(ax=ax, ls='-', color='darkblue')
    ax.legend(bbox_to_anchor=(1, 1), loc="upper left")
    ax.set_ylabel("Power [GW]")
    ax.set_xlabel("")
    fig.tight_layout()
    suffix = (
        "-" + datetime.strptime(str(timeslice), "%m").strftime("%b")
        if timeslice != "all"
        else ""
    )
    path = Path(snakemake.output.operation_area)
    fig.savefig(path.parent / (path.stem + suffix + path.suffix))

In [ ]:
fig, ax = plt.subplots()
total_production = n.snapshot_weightings.generators @ production
total_production.div(1e3).plot.bar(color=colors, ax=ax)
ax.set_ylabel("Total production [TWh]")
ax.set_xlabel("")
fig.savefig(snakemake.output.operation_bar)

In [ ]:
fig, ax = plt.subplots()

production = n.generators_t.p
operational_costs = (
    (production * n.generators.marginal_cost)
    .groupby(carriers, axis=1)
    .sum()
    .rename(columns=n.carriers.nice_name)
).sum()

capital_costs = (
    n.generators.eval("p_nom_opt * capital_cost")
    .groupby(carriers)
    .sum()
    .rename(n.carriers.nice_name)
)

costs = pd.concat([operational_costs, capital_costs], axis=1, keys=["OPEX", "CAPEX"])
costs = costs.reset_index("carrier")


s1 = sns.barplot(x="carrier", y="CAPEX", data=costs, alpha=0.6, ax=ax, palette=colors)
s2 = sns.barplot(
    x="carrier", y="OPEX", data=costs, ax=ax, bottom=costs["CAPEX"], palette=colors
)

ax.set_xlabel("")
ax.set_ylabel("CAPEX & OPEX [bn€]")
fig.savefig(snakemake.output.cost_bar)